In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [51]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str))) #set 중복 제거
print(char_vocab)

['!', 'a', 'e', 'l', 'p']


In [52]:
vocab_size = len(char_vocab)
print(f'문자 집합의 크기:{vocab_size}')

문자 집합의 크기:5


In [53]:
input_size = vocab_size
hidden_size = 5
output_size = 5
lr = 0.1

In [54]:
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [55]:
index_to_char = {} #나중에 예측 결과를 다시 문자 시퀀스로 보기 위해서 반대로도 만들어 놓음
for key,value in char_to_index.items():
  index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [56]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data) #a,p,p,l,e
print(y_data) #p,p,l,e,!

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [57]:
#파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력 받음. 그렇기 떄문에 배치 차원을 추가해준다
#배치 차원 추가
#텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있음

x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [58]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data] #총 행렬의 크기 5x5로 만들어라 
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [59]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print(f'훈련 데이터의 크기:{X.shape}')
print(f'레이블의 크기:{Y.shape}')

훈련 데이터의 크기:torch.Size([1, 5, 5])
레이블의 크기:torch.Size([1, 5])


In [60]:
class Net(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(Net,self).__init__()
    self.rnn = nn.RNN(input_size,hidden_size,batch_first=True) #RNN 셀 구현

    self.fc = nn.Linear(hidden_size,output_size,bias=True) #출력층 구현

  def forward(self,x):
    x,_status = self.rnn(x)
    x = self.fc(x)
    return x

In [61]:
net = Net(input_size,hidden_size,output_size)

In [62]:
criterion = nn.CrossEntropyLoss()
oprimizer = optim.Adam(net.parameters(),lr)

outputs = net(X)
print(outputs)
print('')
print(outputs.shape) #배치 차원, 시점(timesteps), 출력의 크기 

tensor([[[ 1.9191e-02, -4.2972e-01,  2.9179e-02, -1.6109e-01,  1.4142e-01],
         [-1.2390e-02, -5.2069e-01, -6.0742e-01, -4.8536e-01, -8.3924e-02],
         [-8.8123e-02, -5.3764e-01, -6.3862e-01, -4.6396e-01, -8.3717e-02],
         [-8.0806e-02, -5.1205e-01, -6.1621e-01, -4.3224e-01,  3.1076e-04],
         [-1.7154e-01, -5.4297e-01, -3.9047e-01, -2.1034e-01,  1.1845e-01]]],
       grad_fn=<AddBackward0>)

torch.Size([1, 5, 5])


In [63]:
print(outputs.view(-1,input_size).shape) #2차원 텐서로 변환 

torch.Size([5, 5])


In [64]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr)

In [66]:
for i in range(10):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1,input_size), Y.view(-1)) #view를 하는 이유 Batch차원 제거를 위해

  loss.backward()
  optimizer.step()

  #모델이 실제 어떻게 예측했는지를 확인하기 위한 코드
  result = outputs.data.numpy().argmax(axis=2)
  result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
  print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.6042366027832031 prediction:  [[4 0 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  p!ppp
1 loss:  1.4144153594970703 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.3271619081497192 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
3 loss:  1.2430455684661865 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
4 loss:  1.139857292175293 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
5 loss:  1.0199990272521973 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
6 loss:  0.8814024925231934 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
7 loss:  0.7277898788452148 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
8 loss:  0.6384915113449097 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
9 loss:  0.48990267515182495 prediction:  [[4 4 4 2 0]] 

## 더 많은 문장으로 했을때는?

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim 

In [68]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [69]:
len(sentence)

180

In [70]:
char_set = list(set(sentence)) #중복을 제거한 문자 집합
char_dic = {c: i for i, c in enumerate(char_set)} #각 문자에 정수 인코딩
print(char_dic)

{'i': 0, 'a': 1, 'w': 2, 'r': 3, '.': 4, 'u': 5, 'y': 6, "'": 7, 'k': 8, ',': 9, 'g': 10, 'c': 11, 'n': 12, 'h': 13, 'b': 14, 's': 15, 'd': 16, 'o': 17, 'l': 18, 'e': 19, 'f': 20, 't': 21, 'p': 22, ' ': 23, 'm': 24}


In [71]:
dic_size = len(char_dic)
print(f'문자 집합의 크기:{dic_size}')

문자 집합의 크기:25


In [72]:
hidden_size = dic_size
sequence_length = 10
lr = 0.1

In [73]:
x_data = []
y_data = []

for i in range(0,len(sentence) - sequence_length):
  x_str = sentence[i: i + sequence_length]
  y_str = sentence[i+1: i + sequence_length + 1]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str]) # x str to index
  y_data.append([char_dic[c] for c in y_str]) # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [74]:
print(x_data[0]) #if you wan에 해당
print(y_data[0]) # f you want에 해당
#한 칸씩 쉬프트로 된 시퀀스 

[0, 20, 23, 6, 17, 5, 23, 2, 1, 12]
[20, 23, 6, 17, 5, 23, 2, 1, 12, 21]


In [75]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] #x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [76]:
print(f'훈련 데이터의 크기:{X.shape}')
print(f'레이블의 크기:{Y.shape}')

훈련 데이터의 크기:torch.Size([170, 10, 25])
레이블의 크기:torch.Size([170, 10])


In [77]:
print(X[0])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,

In [78]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [79]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

In [80]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr)

In [81]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [82]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


In [83]:
for i in range(30):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

b,,,bn,,nbbn,b,,n,,,bnb,,nn,,nbb,,b,,nnbnnbnn,n,n,,b,n,,n,b,b,nb,,nnbb,bb,,,b,,,bb,,b,n,nnb,nnnnb,bn,nb,b,,,bbn,,nn,b,nb,n,b,n,n,b,,nnb,b,,bb,b,b,b,nn,,,bnn,,bnnnnbbnb,,,b,,,n,,n,
                                                                                                                                                                                   
 bwo lbo  lbo lbmtl oolello lollle   lleo le  lbe lle  llleo lollll  lo  lbo  bo  leeo llb  lee to llllol lo  leoo ebo  lbo o lollo lleo le  leo  eo  lb  bl  lb lbd lle lllo   lll
e to  to   to  o to to  e   to  te     to t   to to       to to to   to   o e to      e to to   to  e toe to  o to  o  o   to  o to   to  o t to  to  to  o  t    to to to toe to  
t toa to nit   a io t a t   t a to ns tta tn  t  toa .  n taetn .ot  tneetooh tna t tnni.mttae sta ns t a tot t tns w t t   a te t tn ta th w t a tnanw'. a teana tnitottn .otnt t 
totoa tand tonthsoa todotoa tod'toaii'tan toam tototo tonitoniplaoio ton'laos ton ttoniito tae'ltosm

## Embedding 사용!!

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim

In [85]:
sentence = "Repeat is the best medicine for memory".split()
vocab = list(set(sentence))
print(vocab)
#우리가 만들 RNN은 "Repeat is the best medicine for"을 입력 받으면 "is the best medicine for memory"를 출력하는 RNN

['for', 'Repeat', 'best', 'the', 'is', 'memory', 'medicine']


In [86]:
word2index = {tkn:i for i,tkn in enumerate(vocab,1)} #단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)

{'for': 1, 'Repeat': 2, 'best': 3, 'the': 4, 'is': 5, 'memory': 6, 'medicine': 7, '<unk>': 0}


In [87]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'Repeat', 3: 'best', 4: 'the', 5: 'is', 6: 'memory', 7: 'medicine', 0: '<unk>'}


In [88]:
#우리가 사용할 word2index단어장
def build_data(sentence,word2index):
  encoded = [word2index[token] for token in sentence]
  input_seq, label_seq = encoded[:-1],encoded[1:]
  input_seq = torch.LongTensor(input_seq).unsqueeze(0)
  label_seq = torch.LongTensor(label_seq).unsqueeze(0)
  return input_seq,label_seq

In [89]:
X,Y = build_data(sentence,word2index)
print(X)
print(Y)

tensor([[2, 5, 4, 3, 7, 1]])
tensor([[5, 4, 3, 7, 1, 6]])


In [90]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) 

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [91]:
vocab_size = len(word2index) 
input_size = 5      
hidden_size = 20   

In [92]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(params=model.parameters())

In [93]:
#(시퀀스의 길이, 은닉층의 크기)
output = model(X)
print(output)
print('')
print(output.shape)

tensor([[ 0.2609,  0.2517, -0.4234,  0.1609,  0.2980, -0.6366,  0.0351, -0.1422],
        [ 0.0825,  0.1859, -0.1430, -0.2159,  0.4267, -0.5054,  0.1280, -0.1824],
        [ 0.4297,  0.1642, -0.1339,  0.0602,  0.0755, -0.3480,  0.0546, -0.1096],
        [ 0.2407,  0.2043, -0.0137, -0.1367,  0.3346, -0.2834,  0.0984, -0.1080],
        [ 0.3557,  0.1780,  0.1426,  0.0720,  0.1709, -0.2613, -0.0570, -0.2877],
        [ 0.1746,  0.1438,  0.0553, -0.2467,  0.1375,  0.2544, -0.1970, -0.2312]],
       grad_fn=<ViewBackward>)

torch.Size([6, 8])


In [94]:
decode = lambda y: [index2word.get(x) for x in y]

In [96]:
for step in range(201):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_function(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 1.9008 
Repeat the the best the for the

[41/201] 1.3098 
Repeat the the best medicine for memory

[81/201] 0.6863 
Repeat is the best medicine for memory

[121/201] 0.3627 
Repeat is the best medicine for memory

[161/201] 0.1997 
Repeat is the best medicine for memory

[201/201] 0.1181 
Repeat is the best medicine for memory



Reference: https://wikidocs.net/64765 를 참조하였습니다.
